Code based on https://www.kaggle.com/thebrownviking20/everything-you-can-do-with-a-time-series
also view this https://www.kaggle.com/learn/intro-to-machine-learning


In [20]:
!{sys.executable} -m pip install xgboost 
!{sys.executable} -m pip install tensorflow==2.4
!{sys.executable} -m pip install kerastuner

import numpy as np
import pandas as pd
import time
import string
import re
import math

from collections import Counter

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LinearRegression, Ridge, Lasso, TweedieRegressor,HuberRegressor
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import mean_squared_error as mse


import xgboost as xgb

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau

import kerastuner as kt
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

from bokeh.plotting import figure
from bokeh.models import ColumnDataSource
from bokeh.io import curdoc, show, output_notebook
output_notebook()

import nltk
from nltk import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

import spacy
nlp = spacy.load('en_core_web_lg')

import transformers
from transformers import BertTokenizer, TFBertModel, RobertaTokenizer, TFRobertaModel
print('--------------------importing completed--------------------------------')

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 394.7 MB 5.7 kB/s eta 0:00:01   |▌                               | 6.8 MB 2.5 MB/s eta 0:02:39     |█▏                              | 14.5 MB 521 kB/s eta 0:12:09     |██                              | 24.6 MB 2.0 MB/s eta 0:03:10     |██▋                             | 31.8 MB 1.9 MB/s eta 0:03:12     |██▋                             | 32.7 MB 2.7 MB/s eta 0:02:16     |████▍                           | 54.6 MB 3.0 MB/s eta 0:01:56     |████▋                           | 56.7 MB 3.6 MB/s eta 0:01:34     |█████▎                          | 64.9 MB 2.9 MB/s eta 0:01:55     |█████▋                          | 69.5 MB 2.7 MB/s eta 0:02:03     |██████                          | 73.9 MB 3.4 MB/s eta 0:01:36     |██████                          | 74.7 MB 2.4 MB/s eta 0:02:15     |███████▉                

  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.34.1
    Uninstalling grpcio-1.34.1:
      Successfully uninstalled grpcio-1.34.1
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.5.0
    Uninstalling tensorflow-estimator-2.5.0:
      Successfully uninstalled tensorflow-estimator-2.5.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
  Attempting uninstall: gast
    Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Successfully uninstalled gast-0.4.0
Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement kerastuner (from versions: none)
ERROR: No matching distribution found for kerastuner


ModuleNotFoundError: No module named 'kerastuner'

In [ ]:
df_train = pd.read_csv('/kaggle/input/commonlitreadabilityprize/train.csv')
df_test = pd.read_csv('/kaggle/input/commonlitreadabilityprize/test.csv')
df_submission = pd.read_csv('/kaggle/input/commonlitreadabilityprize/sample_submission.csv')

print(" Training dataset shape : " + str(df_train.shape))
print(" Testing dataset shape : " + str(df_test.shape))

In [ ]:
df_train.head()

In [ ]:
df_train['excerpt'][0]

In [ ]:
df_test.head()

In [ ]:
df_submission.head()

## Exploratory analysis

In [ ]:
# Only url_legal and license columns appear to be having missing values
df_train.isnull().sum()

In [ ]:
df_test.isnull().sum()

In [ ]:
# Examples with the 5 lowest target values
display(df_train.sort_values(by=['target']).head())

In [ ]:
# Examples with the 5 highest target values
display(df_train.sort_values(by=['target'], ascending=False).head())

In [ ]:
# View target distribution
fig, ax = plt.subplots(1, 1, figsize=(20, 6))
sns.distplot(df_train['target'], ax=ax, color ='green')
plt.show()

In [ ]:
# View std_error distribution
fig, ax = plt.subplots(1, 1, figsize=(20, 6))
#sns.distplot(df_train['standard_error'], ax=ax, color ='green')
plt.show()

In [ ]:
def msv_1(data, color = 'yellow', edgecolor = 'black', height = 3, width = 15):
    plt.figure(figsize = (width, height))
    percentage = (data.isnull().mean()) * 100
    percentage.sort_values(ascending = False).plot.bar(color = color, edgecolor = edgecolor)

    plt.title('Missing values percentage per column', fontsize=20, weight='bold' )
    plt.xlabel('Columns', size=15, weight='bold')
    plt.ylabel('Missing values percentage')
    plt.yticks(weight ='bold')
    
    return plt.show()
msv_1(df_train, color=sns.color_palette('flare',15))

In [ ]:
# Count number of words in excerpts and maximum count
count = df_train['excerpt'].str.split().str.len()
print("Number of words in excerpts:\n",count)
print("Max word count from excerpt: ", max(count))